# Introduction

This notebook aims to showcase the application of our model for predicting equivariant dielectric tensors of inorganic materials.

PFP (see details at https://tech.preferred.jp/en/blog/development-of-universal-neural-network-for-materials-discovery/) is a neural network potential model that incorporates scalar, equivariant vector, and tensor features in each node. We demonstrate that the equivariant features learned in the pre-trained PFP can be effectively utilized to predict other tensorial properties, yielding promising accuracy even with limited available data. In this notebook, we utilize the dielectric constants (~6.6k data) extracted from the Materials Project (https://next-gen.materialsproject.org/) as an illustrative example.

## 1. Model training

A specifically-designed equivariant readout module is utilized for the dielectric tensor. This module takes into account the scalar, vector, and tensor features of nodes, as well as the scalar and vector features of edges, which are extracted from the pre-trained PFP. By combining and processing these inputs, the module predicts the equivariant 3 by 3 dielectric constants.

In [1]:
import sys
sys.path.append("../")

import os
import argparse
import yaml
from sklearn.model_selection import train_test_split

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from pytorch_lightning.loggers import CSVLogger, WandbLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

from utils.training import DielectricModule
from data.dataset import JSONDataset
from models.equivariant_model import GatedEquivariantModel
from confidential.models import CombinedModel
from confidential.utils import load_pfp, collate_fn_dict


/mnt/nfs-mnj-archive-02/user/pe_zmao/micromamba/envs/pfp/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load the configuration for model training. This configuration is used to train a model for the electronic contribution of dielectric tensors.

In [2]:
with open('../scripts/train_config.yaml','r') as f:
    configs = yaml.safe_load(f)

# dimensionality of each feature of PFP
ns_feat = configs['Model'].pop('ns_feat')   # node scalar feature
nv_feat = configs['Model'].pop('nv_feat')   # node vector feature
nt_feat = configs['Model'].pop('nt_feat')   # node tensor feature
es_feat = configs['Model'].pop('es_feat')   # edge scalar feature
ev_feat = configs['Model'].pop('ev_feat')   # edge vector feature

configs

{'Seed': 3,
 'Train': {'target': 'electronic',
  'num_workers': 12,
  'dataset': '../data/mp_dielectric.json',
  'batch': 64,
  'epoch': 5,
  'patience': 200,
  'lr': 0.0001,
  'accelerator': 'gpu',
  'device': [3],
  'save_path': '../confidential/checkpoints/',
  'gradient_clip': 2.0},
 'Model': {'pfp_layer': 3,
  'train_pfp': False,
  'latent_feat': 64,
  'n_gate_layers': 2,
  'dropout_rate': 0.0,
  'residual': True,
  'gate_sigmoid': True,
  'mlp_layer': 3,
  'integrate_es_ev': True,
  'integrate_nv_nt': True,
  'apply_mask': False}}

In [3]:
train_config = configs.pop("Train")
model_config = configs.pop("Model")

Set the random seed for code reproducibility.

In [4]:
seed = configs.pop("Seed")
torch.manual_seed(seed)

Prepare data for model training

In [5]:
dataset_file = train_config.pop("dataset")
target = train_config.pop("target")
dataset = JSONDataset(json_file = dataset_file,target = target)

keys = sorted(dataset.keys)
print(f"{len(keys)} data are prepared for training!")

# split dataset
train_keys, test_keys = train_test_split(keys, test_size=0.1, random_state=seed)
train_keys, val_keys = train_test_split(train_keys, test_size=0.1/0.9, random_state=seed)

# prepare dataloader
train_set = JSONDataset(dataset_file, target = target, keys = train_keys)
val_set = JSONDataset(dataset_file, target = target, keys=val_keys)
test_set = JSONDataset(dataset_file, target = target, keys=test_keys)

batch_size = train_config.pop("batch")
num_workers = train_config.pop("num_workers")
train_loader = DataLoader(train_set, batch_size=batch_size, collate_fn=collate_fn_dict, shuffle=True, num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(val_set, batch_size=batch_size, collate_fn=collate_fn_dict, shuffle=False, num_workers=num_workers, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=batch_size, collate_fn=collate_fn_dict, shuffle=False, num_workers=num_workers, pin_memory=True)

6648 data are prepared for training!


Load the pre-trained PFP model and freeze its parameters

In [6]:
pfp_layer = model_config.pop("pfp_layer")
pfp_wrapped = load_pfp(load_parameters=True, return_layer=pfp_layer)

train_pfp = model_config.pop("train_pfp")
if train_pfp:
    # Initalize the parameters if PFP is also trained
    pfp_wrapped.pfp.reset_parameters()
else:
    # Otherwise, freeze parameters in PFP
    for param in pfp_wrapped.parameters():
        param.requires_grad = False

Initalize the equivariant readout model and combine it with the pre-trained PFP

In [7]:
tensorial_model = GatedEquivariantModel(
    ns_feat=ns_feat,
    nv_feat=nv_feat,
    nt_feat=nt_feat,
    es_feat=es_feat,
    ev_feat=ev_feat,
    **model_config)
model = CombinedModel(pfp_wrapped, tensorial_model)

Set the Trainer in the pytorch lightning package.

In [8]:
lr = train_config.pop("lr")
pl_module = DielectricModule(model, learning_rate=lr, optimizer='adam')
outdir = train_config.pop("save_path")

checkpoint_callback = ModelCheckpoint(
    save_top_k=1,
    monitor="val_loss",
    mode="min",
    dirpath=f"{outdir}/pl_checkpoints/",
    filename="eps-{epoch:02d}-{val_loss:.2f}",
)

patience = train_config.pop("patience")
earlystopping_callback = EarlyStopping("val_loss", mode="min", patience=patience)

epoch = train_config.pop("epoch")
accelerator = train_config.pop("accelerator")
devices = train_config.pop("device")
clip_val = train_config.pop("gradient_clip")

trainer = pl.Trainer(
    max_epochs=epoch, 
    accelerator=accelerator, 
    devices=devices, 
    callbacks=[checkpoint_callback, earlystopping_callback],
    gradient_clip_val=clip_val,
    enable_model_summary=False)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/mnt/nfs-mnj-archive-02/user/pe_zmao/micromamba/envs/pfp/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(


Train the model. For showcase purposes, we only train for 5 epochs.

In [9]:
trainer.fit(model=pl_module, train_dataloaders=train_loader, val_dataloaders=val_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [4,3,0,7,5,6,2,1]


Epoch 4: 100%|██████████| 84/84 [00:07<00:00, 10.95it/s, v_num=0, val_loss=25.40, val_trace_loss=6.870, val_diag_loss=25.20, val_off_loss=0.075, val_tensor_loss=25.40, train_loss=39.70, train_trace_loss=10.60, train_diag_loss=39.10, train_off_loss=0.303, train_tensor_loss=39.70, lr=0.0001] 

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 84/84 [00:07<00:00, 10.66it/s, v_num=0, val_loss=25.40, val_trace_loss=6.870, val_diag_loss=25.20, val_off_loss=0.075, val_tensor_loss=25.40, train_loss=39.70, train_trace_loss=10.60, train_diag_loss=39.10, train_off_loss=0.303, train_tensor_loss=39.70, lr=0.0001]


## 2. Evaluation of model performance on the test set

In [10]:
trainer.test(model=pl_module, dataloaders=test_loader, ckpt_path='../scripts/pl_checkpoints/eps-epoch=79-val_loss=11.27.ckpt')

Restoring states from the checkpoint path at ../scripts/pl_checkpoints/eps-epoch=79-val_loss=11.27.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [4,3,0,7,5,6,2,1]
Loaded model weights from the checkpoint at ../scripts/pl_checkpoints/eps-epoch=79-val_loss=11.27.ckpt


Testing DataLoader 0: 100%|██████████| 11/11 [00:00<00:00, 16.43it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
          loss              0.4318067133426666
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'loss': 0.4318067133426666}]

## 3. Predict dielectric tensors based on structures

In [12]:
pl_module = DielectricModule.load_from_checkpoint(
    '../scripts/pl_checkpoints/eps-epoch=79-val_loss=11.27.ckpt',
    map_location = "cuda:0",
    model = model,
)
pred = pl_module.predict_atoms_from_file('../scripts/test_structures/Na14Mn2O9.cif')

print(pred)

tensor([[[ 3.3504e+00, -4.8901e-08,  2.2176e-07],
         [-4.8901e-08,  3.3504e+00, -9.7048e-08],
         [ 2.2176e-07, -9.7048e-08,  3.3882e+00]]])
